In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from datetime import datetime
import pubmed as pm
import pandas as pd
import time
import os

In [19]:
pdb_path = f'{os.getcwd()}/PDBs'
log_path = f'{os.getcwd()}/log'
url = 'http://webs.iiitd.edu.in/raghava/cancerppd/browse_peptide.php'

In [3]:
data = {
    'CancerPPD ID': [],
    'PMID': [],
    'YEAR': [],
    'SEQUENCE': [],
    'NAME': [],
    'LENGTH': [],
    'LINEAR/CYCLIC': [],
    'CHIRAL': [],
    'CHEM-MOD': [],
    'C-ter MOD': [],
    'N-ter MOD': [],
    'NATURE': [],
    'ORIGIN': [],
    'CELL LINE': [],
    'CANCER TYPE': [],
    'ASSAY': [],
    'ACTIVITY': [],
    'TESTING TIME': [],
    'TISSUE EFFECTED': [],
    'PATENTS': [],
    'SMILES': []
}

In [33]:
driver = webdriver.Chrome()

In [5]:
count_errors = []
error_line = ''
count = 1 # max=623
while True:
    driver.get(url)
    time.sleep(2)
    
    #Show all peptides
    select = Select(driver.find_element(By.XPATH, '/html/body/table[4]/tfoot/tr/td/select'))
    select.select_by_value('1000')
    
    #Show details from one peptide
    try:
        pep = driver.find_element(By.XPATH, f'/html/body/table[4]/tbody/tr[{count}]/td[1]/a')
        error_line = f'{count}, {pep.text}'
        pep.click()
        time.sleep(2)
    except:
        print('Finished')
        break
        
    #Capture data
    try:
        table = driver.find_element(By.XPATH, '/html/body/div[4]/div[1]/table/tbody')
        elements = table.find_elements(By.TAG_NAME, 'tr')
        for element in elements:
            props = element.find_elements(By.TAG_NAME, 'td')
            index = 0
            for key in data.keys():
                data[key].append(props[index].text)
                index += 1
    except:
        count_errors.append(error_line)
    finally:
        count += 1

Finished


In [34]:
df = pd.DataFrame(data)
#df = pd.read_csv(f'{pdb_path}/CancerPPD_releases/CancerPPD_2023_01_18.csv')

In [37]:
# Get DOI from PubMed
doi_list = []

#for pmid in data['PMID']:
for pmid in df['PMID']:
    if pmid != 'None':
        if len(pmid.split(',')) > 1:
            [pmid1, pmid2] = pmid.split(', ')
            doi1 = pm.get_doi_from_pmid(driver, pmid1)
            doi2 = pm.get_doi_from_pmid(driver, pmid2)
            doi_list.append([doi1, doi2])
        else:
            doi = pm.get_doi_from_pmid(driver, pmid)
            doi_list.append(doi)
    else:
        doi_list.append('Not Found')
    
df['DOI'] = doi_list

In [41]:
driver.close()

In [26]:
#Write log file with failed peptides download 
dt = datetime.now()
with open(f'{log_path}/CancerPPD_errors_{dt.strftime("%Y_%m_%d_%H_%M")}.log', 'w', encoding='utf8') as file:
    print('failed peptides download:', len(count_errors))
    for value in count_errors:
        file.write(value)
        print(value.split(', '))

failed peptides download: 13
['126', 'FALALKALKK']
['335', 'I-C28H46N4O6,II-C27H44N4O6,III-C26H42N4O6']
['376', 'KKβ2,2WKK']
['474', 'KWβ2,2KK']
['516', 'NotAvailable']
['616', 'YHWYGYTPQNVIGGGKLlLKlLkkLLKlLKKK']
['617', 'YKQCHKKGGHCFPKEKICLPPSSDFGKMDCRWRWKCCKKGSG']
['618', 'YPFPG']
['619', 'YRA*G']
['620', 'YRWYGYTPQNVIGGGKLlLKlLkkLLKlLKKK']
['621', 'YSA*']
['622', 'YSA*G']
['623', 'YSL']


In [40]:
dt = datetime.now()
df.to_csv(f'{pdb_path}/CancerPPD_releases/CancerPPD_{dt.strftime("%Y_%m_%d")}.csv', index=False)
df.to_csv(f'{pdb_path}/CancerPPD.csv', index=False)

In [42]:
df

,PMID,YEAR,SEQUENCE,NAME,LENGTH,LINEAR/CYCLIC,CHIRAL,CHEM-MOD,C-ter MOD,N-ter MOD,...,ORIGIN,CELL LINE,CANCER TYPE,ASSAY,ACTIVITY,TESTING TIME,TISSUE EFFECTED,PATENTS,SMILES,DOI
0,17787026,2007,(K-Aib-C(CH2CO-2'-Pac))2,Dimer of peptide-paclitaxel conjugate,3,Linear,L,"Aib = 1-amino-isobutyric acid ,Pac = Paclitaxel",Amidation,Acetylation,...,Synthetic peptide paclitaxel conjugate,HeLa,Cervical Cancer,MTT/MTS assay,IC50 =1.31nM,24-h,Cervix,None,NaN,https://doi.org/10.1002/psc.899
1,17787026,2007,(K-Aib-C(CH2CO-2'-Pac))3,Dimer of peptide-paclitaxel conjugate,3,Linear,L,"Aib = 1-amino-isobutyric acid ,Pac = Paclitaxel",Amidation,Acetylation,...,Synthetic peptide paclitaxel conjugate,DU,Prostate Cancer,MTT/MTS assay,IC50 =2.85nM,24-h,Prostate,None,NaN,https://doi.org/10.1002/psc.899
2,17787026,2007,(K-Aib-C(CH2CO-2'-Pac))3,Trimer of peptide-paclitaxel conjugate,3,Linear,L,"Aib = 1-amino-isobutyric acid ,Pac = Paclitaxel",Amidation,Acetylation,...,Synthetic peptide paclitaxel conjugate,HeLa,Cervical Cancer,MTT/MTS assay,IC50 =1.70nM,24-h,Cervix,None,NaN,https://doi.org/10.1002/psc.899
3,17787026,2007,(K-Aib-C(CH2CO-2'-Pac))3,Trimer of peptide-paclitaxel conjugate,3,Linear,L,"Aib = 1-amino-isobutyric acid ,Pac = Paclitaxel",Amidation,Acetylation,...,Synthetic peptide paclitaxel conjugate,DU,Prostate Cancer,MTT/MTS assay,IC50 =3.88nM,24-h,Prostate,None,NaN,https://doi.org/10.1002/psc.899
4,17787026,2007,(K-Aib-C(CH2CO-2'-Pac))3,Trimer of peptide-paclitaxel conjugate,3,Linear,L,"Aib = 1-amino-isobutyric acid ,Pac = Paclitaxel",Amidation,Acetylation,...,Synthetic peptide paclitaxel conjugate,MCF-7,Breast Cancer,MTT/MTS assay,IC50 =3.00nM,24-h,Breast,None,NaN,https://doi.org/10.1002/psc.899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5063,24176852,2013,YGRKKRRQRRRREADFFWSLCTADMS,KilerFLIP,26,linear,L,None,Free,Free,...,NaN,Jurkat,Blood Cancer,Not Available,NaN,NaN,Blood,None,N[C@@H](Cc1ccccc1)C(=O)NCC(=O)N[C@@H](CCCN=C)C...,https://doi.org/10.1038/cddis.2013.401
5064,12417587,2003,YGRKKRRQRRR,Tat (47-57),11,Linear,L,None,Free,Free,...,Synthetic,HeLa,Cervical Cancer,MTT/MTS assay,at 1 µM 90-100% viablity,6-h,Cervix,None,N[C@@H](Cc1ccccc1)C(=O)NCC(=O)N[C@@H](CCCN=C)C...,https://doi.org/10.1074/jbc.M208762200
5065,12417587,2003,YGRKKRRQRRR,Tat (47-57),11,Linear,L,None,Free,Free,...,Synthetic,HeLa,Cervical Cancer,MTT/MTS assay,at 10 µM 80% viablity,6-h,Cervix,None,N[C@@H](Cc1ccccc1)C(=O)NCC(=O)N[C@@H](CCCN=C)C...,https://doi.org/10.1074/jbc.M208762200
5066,12417587,2003,YGRKKRRQRRR,Tat (47-57),11,Linear,L,None,Free,Free,...,Synthetic,HeLa,Cervical Cancer,MTT/MTS assay,at 100 µM 60% viablity,6-h,Cervix,None,N[C@@H](Cc1ccccc1)C(=O)NCC(=O)N[C@@H](CCCN=C)C...,https://doi.org/10.1074/jbc.M208762200
